## SAMPLE CLASS
_Represent a sample from a dataset_

### Methods
 Methods | Descriptions 
---------|--------------
**\_\_init\_\_(self, dataset\_location, site, file)** |  Initialise class based on absolute location of the dataset, the name of the site, and the data file name
**get_file(self) -\> File name** | Return the data file name
**frread(self)->** | Extract the raw data from the vr2 data file
**vr2_to_panda(self)** | Extract the data and store it into a Panda dataframe


In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import datetime
from math import ceil, sqrt, floor, gamma
from scipy import signal, stats, fftpack 
from sklearn import preprocessing, mixture
from PIL import Image
import scipy.stats as stats
from matplotlib import image, patches, pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# import python library
sys.path.append(os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'py'))

from dataset import Dataset
# from database import Database


class Sample(Dataset):
    ''''''
    # Attributes
    __freq, __time, __spec = None, None, None
    __output = None
    __img = None
    __figsize = None
    
    # spectrogram default parameters
    __NPERSEG = 256        # Length of each segment. Defaults to None, but if window is str or tuple, is set to 256, 
                            # and if window is array_like, is set to the length of the window.
    __NOVERLAP = 0        # Number of points to overlap between segments. If None, noverlap = nperseg // 8. Defaults to None.
    __NFFT = 256         # Length of the FFT used, if a zero padded FFT is desired. If None, the FFT length is nperseg. Defaults to None.
    __DETREND = 'constant'      # Specifies how to detrend each segment. If detrend is a string, it is passed as the type argument to 
                            # the detrend function. If it is a function, it takes a segment and returns a detrended segment. 
                            # If detrend is False, no detrending is done. Defaults to ‘constant’ [‘linear’, ‘constant’]
    __SCALING = 'spectrum' # Selects between computing the power spectral density (‘density’) where Sxx has units of V**2/Hz and 
                            # computing the power spectrum (‘spectrum’) where Sxx has units of V**2, if x is measured in V and fs is measured in Hz. Defaults to ‘density’.
    __MODE = 'magnitude'         # Defines what kind of return values are expected. Options are [‘psd’, ‘complex’, ‘magnitude’, ‘angle’, ‘phase’]. 
                            # ‘complex’ is equivalent to the output of stft with no padding or boundary extension. 
                            # ‘magnitude’ returns the absolute magnitude of the STFT. 
                            # ‘angle’ and ‘phase’ return the complex angle of the STFT, with and without unwrapping, respectively.

    
    # Initializer
    def __init__(self, dataset_location, site, file):
        self.__file = file
        super().__init__(dataset_location, site)
    
    ##########################################################
    # DATA EXTRACTION
    def frread(self):
        """ This is a rough translation of frread.m from J. Lichtenberger for the
        stereo=True case, i.e. we assume orthogonal loop antenna.
        inputs
            fname (string): File name path to the .vr2 file to load
        outputs
            wh (ndarray): 2xN array with the two traces in the first and second rows.
        """
        # open file for reading
        fid = open(os.path.join(self.get_dataset_location(), self.get_site(), self.get_site()+'_data', self.__file), 'rb')
        # get data from file - 16-bit signed integers
        dat = np.fromfile(fid, dtype=np.int16)
        # length of one frame
        """ Each frame consists of 4103 words (each word consists of 16 bits, 
        two bytes in LSB format, i.e. a frame is 8206 bytes long). 
        A frame consists of a 7 words (14 bytes) long header and a 4096 
        words long data block."""
        frLen = 4103  ## not sure how this is determined
        # number of frames to read
        nFrameRead = len(dat) / frLen
        # data length of frame
        adatlen = 2048
        # length of data set
        N = int(nFrameRead * adatlen)
        wh = np.zeros((N, 2), dtype=float)
        # for every frame
        for i in np.arange(0, nFrameRead, dtype=int):
            # indices for first component
            i1 = np.arange(7 + i * frLen, (i + 1) * frLen, 2, dtype=int)
            # indices for second component
            i2 = np.arange(8 + i * frLen, (i + 1) * frLen + 0, 2, dtype=int)
            ii = np.arange(i * adatlen, (i + 1) * adatlen, dtype=int)
            wh[ii, 0] = dat[i1]
            wh[ii, 1] = dat[i2]
        # first 7 bits of the status word of each frame, in this case the first frame
        div = dat[6]&0xF
        # sample frequency
        fs = 200e3/(div+1)
        return wh
    
    def vr2_to_panda(self):
        """Extract the data from a file a store it as a Panda DataFrame
        inputs
            fname    file name
            site     name of the site where data was collected
        outputs 
            whdf     dataframe containing the signal received by the NS and EW pointitng
                        orthogonal loop antennas
            fs       sampling frequency
        """
        # read vr2 file
        wh = self.frread()

        # CONSTANTS
        # Sampling frequency (20kHz for SANAE, 40kHz for MARION )
        if self.get_site()=='marion':
            fs = 4e4
        elif self.get_site()=='sanae':
            fs = 2e4
        else:
            fs = 4e4
        # time step in microseconds (for dataframe index)
        dt = 1e6 / fs

        # Set the date/time format in the filename
        # dtFormat = '%Y-%m-%dUT%H_%M_%S.%f'
        dtFormat = '%Y-%m-%dUT%H:%M:%S.%f'

        # Set up pandas dataframe
        # Start time
        t0 = pd.datetime.strptime(self.__file[0:27], dtFormat)
        # Number of samples
        Nsamples = len(wh[:, 0])
        # End time
        t1 = t0 + datetime.timedelta(0, 0, Nsamples * dt)
        # Create index
        tindex = pd.date_range(start=t0, end=t1, periods=Nsamples) #freq='50U') # freq = 50us

        # Create pandas data frame from wh
        whdf = pd.DataFrame(index=tindex, data=wh[:, 0], columns=['X'])
        whdf['Y'] = wh[:, 1]
        # The 'X' and 'Y' columns are the signal received by the North/South and
        # East/West pointing orthogonal loop antennas used at Marion and SANAE

        return whdf, fs
    
    ############################################
    # SIGNAL 
    def signal_plot(self, sig=None, EW=True, figsize=None, title=None):
        if sig is None:
            sig, _ = self.vr2_to_panda()
            sig = sig['X'] if EW else sig['Y']
        fig = plt.figure(figsize=figsize) if figsize else plt.figure()
        ax = plt.subplot(111)
        ax.margins(0)
        if title:
            plt.title(title)
        plt.plot(sig.index, sig.values)
        plt.show()
    
    def signal_power_plot(self, sig=None, EW=True, figsize=None, title=None):
        if sig is None:
            sig, _ = self.vr2_to_panda()
            sig = sig['X'] if EW else sig['Y']
        sig = sig**2/sig.size
        fig = plt.figure(figsize=figsize) if figsize else plt.figure()
        ax = plt.subplot(111)
        ax.margins(0)
        if title:
            plt.title(title)
        plt.plot(sig.index, sig.values)
        plt.show()

    def signal_hist(self, EW=True, figsize=None, xlabel='Bins', ylabel=None, title='Signal\'s histogram and fitted gaussian distribution' ):
        '''Plot histrogram of signal'''
        sig = self.frread()[:,0] if EW else self.frread()[:,1]
        sig = np.asarray(sig)
        if figsize:
            fig = plt.figure(figsize=figsize)
        else:
            fig = plt.figure()
        if xlabel:
            plt.xlabel(xlabel)
        if ylabel:
            plt.ylabel(ylabel)
        if title:
            plt.title(title)
        sig = sig.reshape(-1, 1)
        plt.hist(sig, bins=100, histtype='bar', density=True, ec='blue', alpha=0.5)
        g = mixture.GaussianMixture(n_components=1,covariance_type='full')
        g.fit(sig)
        weights, means, covars = g.weights_, g.means_, g.covariances_
        f_axis = sig.copy().ravel()
        f_axis.sort()
        plt.plot(f_axis,weights[0]*stats.norm.pdf(f_axis,means[0],np.sqrt(covars[0])).ravel(), c='red')
        print(weights, means, covars)
        legend = [Patch(facecolor='cornflowerblue', edgecolor='b',label='signal\'s histogram'),
                  Line2D([0], [0], color='red', lw=3, label='gaussian distribution[s]')]
        plt.legend(handles=legend, loc='upper left')
        plt.show()
        
    #################################################
    # GETTERS AND SETTERS
    def get_file(self):
        return self.__file
    def get_spectrogram(self):
        self.assert_spectrogram()
        return self.__spec
    def get_freq(self):
        self.assert_spectrogram()
        return self.__freq
    def get_time(self):
        self.assert_spectrogram()
        return self.__time
    def get_time_res(self):
        self.assert_spectrogram()
        return self.__time_res
    def get_freq_res(self):
        self.assert_spectrogram()
        return self.__freq_res
    def get_time_freq_ratio(self, dec, integer=True):
        self.assert_spectrogram()
        ratio = self.__time_freq_ratio
        ratio = np.round(ratio,dec)
        if integer:
            ratio = int(ratio)
        return ratio+1 if ratio%2==0 else ratio
    def set_figsize(self, freq_len, time_len):
        self.__figsize = (freq_len, time_len)
    def get_figsize(self):
        return self.__figsize 
    def get_image(self):
        return self.__img
    def set_spectrogram(self, spec):
        self.__spec = spec
    ###############################################
    # SPECTROGRAM GENERATION
    def set_spectrogram_params(self, nperseg = __NPERSEG, noverlap = __NOVERLAP,
                            nfft = __NFFT, 
                            detrend = __DETREND,
                            scaling = __SCALING,
                            mode = __MODE):
        '''Always call assert_spectrogram after this function to recompute  self.__spec'''
        self.__NPERSEG = nperseg
        self.__NOVERLAP = noverlap
        self.__NFFT = nfft
        self.__DETREND = detrend
        self.__SCALING = scaling
        self.__MODE = mode
    
    def get_spectrogram_params(self):
        params = {
            'nperseg': self.__NPERSEG, 
            'noverlap': self.__NOVERLAP,
            'nfft': self.__NFFT, 
            'detrend': self.__DETREND,
            'scaling': self.__SCALING,
            'mode': self.__MODE
        }
        return params
    
    def assert_spectrogram(self, force=False, log10=True, kHz=True, abs_val = False):
        '''Compute __spec if None'''
        if self.__spec is None or force:
            self.spectrogram_with_axis(nperseg = self.__NPERSEG, 
                                        noverlap = self.__NOVERLAP,
                                        nfft = self.__NFFT, 
                                        detrend = self.__DETREND,
                                        scaling = self.__SCALING,
                                        mode = self.__MODE, log10=log10, kHz=kHz,abs_val = abs_val)
    
    def spectrogram(self, nperseg = __NPERSEG, noverlap = __NOVERLAP,
                            nfft = __NFFT, 
                            detrend = __DETREND,
                            scaling = __SCALING,
                            mode = __MODE, log10 = True, abs_val = False):
        """Compute spectrogram from vr2 data collected
        outputs
            frequencies, time, and spectrum of the sprectrogram
        """
        whdf, fs = self.vr2_to_panda()
        self.__freq, self.__time, self.__spec = signal.spectrogram(
            whdf.X.values, 
            fs=fs, 
            nperseg=nperseg,
            noverlap=noverlap,
            nfft=nfft, 
            detrend=detrend, 
            scaling=scaling,
            mode=mode)
        if log10:
            self.__spec = np.log10(self.__spec)
        if abs_val:
            self.__spec = abs(self.__spec)
        self.__time_res = self.__time[-1]/len(self.__time)
        self.__freq_res = self.__freq[-1]/len(self.__freq)
        self.__time_freq_ratio = self.__time.shape[0]/self.__freq.shape[0]
        self.__freq_min = self.__freq[0]
        return self.__spec
    
    def spectrogram_with_axis(self, nperseg = __NPERSEG, noverlap = __NOVERLAP,
                            nfft = __NFFT, 
                            detrend = __DETREND,
                            scaling = __SCALING,
                            mode = __MODE, log10 = True, kHz = True, abs_val = False):
        """Compute spectrogram from vr2 data collected
        outputs
            frequencies, time, and spectrum of the sprectrogram
        """
        whdf, fs = self.vr2_to_panda()
        self.__freq, self.__time, self.__spec = signal.spectrogram(
            whdf.X.values, 
            fs=fs, 
            nperseg=nperseg,
            noverlap=noverlap,
            nfft=nfft, 
            detrend=detrend, 
            scaling=scaling,
            mode=mode)
        if kHz:
            self.__freq /= 1e3
        if log10:
            self.__spec = np.log10(self.__spec)
        if abs_val:
            self.__spec = abs(self.__spec)
        self.__time_res = self.__time[-1]/len(self.__time)
        self.__freq_res = self.__freq[-1]/len(self.__freq)
        self.__time_freq_ratio = self.__time.shape[0]/self.__freq.shape[0]
        self.__freq_min = self.__freq[0]
        return self.__freq, self.__time, self.__spec
    
    
    #####################################################
    # SPECTROGRAM PROCESSING 
    def apply_detrend(self, type='linear', bp=1, axis=[-1,0], sig=None):
        if sig is None:
            self.assert_spectrogram()
            for ax in axis:
                self.__spec = signal.detrend(self.__spec, axis=ax, type=type, bp=0)
        #         self.__spec = signal.detrend(self.__spec, axis=0, type=_type, bp=list(range(0, len(spec), int(len(spec)/1))))
        else:
            for ax in axis:
                sig = signal.detrend(sig, axis=ax, type=type, bp=0)
            return sig
        
    def apply_zscore(self,axis=[0,1], sig=None):
        if sig is None:
            self.assert_spectrogram()
            for ax in axis:
                self.__spec = stats.zscore(self.__spec, axis = ax)
        else:
            for ax in axis:
                sig = stats.zscore(sig, axis = ax)
            return sig
    
    def apply_medfilt(self, kernel, iteration=1, sig=None):
        if sig is None:
            self.assert_spectrogram()
            for i in range(iteration):
                self.__spec = signal.medfilt2d(self.__spec, kernel)
        else:
            for i in range(iteration):
                sig = signal.medfilt2d(sig, kernel)
            return sig
        
    def apply_meanfilt(self, kernel, val=1, iteration=1, sig=None):
        if sig is None:
            self.assert_spectrogram()
            for i in range(iteration):
                self.__spec = signal.convolve2d(self.__spec, np.full(shape=kernel, fill_value=val), mode='same')/(kernel[0]*kernel[1])
        else:
            for i in range(iteration):
                sig = signal.convolve2d(sig, np.full(shape=kernel, fill_value=val), mode='same')/(kernel[0]*kernel[1])
            return sig
        
    def apply_slice(self, lower_freq, upper_freq, sig=None):
        if sig is None:
            self.assert_spectrogram()
            lower_freq_ix, upper_freq_ix = ceil(lower_freq/self.get_freq_res()),ceil(upper_freq/self.get_freq_res())
            self.__spec = self.__spec[lower_freq_ix:upper_freq_ix,:]
            self.__freq = self.__freq[lower_freq_ix:upper_freq_ix]
            self.__freq_min = self.__freq[0]
#             print('slice',(lower_freq, upper_freq), (self.get_freq_res()), (lower_freq_ix,upper_freq_ix), 
#                   (int(lower_freq/self.get_freq_res()),int(upper_freq/self.get_freq_res())))
        else:
            sig = sig[int(lower_freq/self.get_freq_res()):int(upper_freq/self.get_freq_res()),:]
            return sig
            
    
    def apply_transform(self, transforms, params, sig=None):
        for transform, param in zip(transforms, params):
            if transform=='detrend':
                sig = self.apply_detrend(type=param[0],sig=sig)
            elif transform=='zscore':
                sig = self.apply_zscore(sig=sig) 
            elif transform=='medfilt':
                sig = self.apply_medfilt(kernel=param[0],sig=sig) 
            elif transform=='meanfilt':
                sig = self.apply_meanfilt(kernel=param[0],sig=sig)
            elif transform=='slice':
                sig = self.apply_slice(lower_freq=param[0],upper_freq=param[1],sig=sig)
            elif transform=='scale':
                sig = self.apply_min_max_scale(minimum=param[0],maximum=param[1],sig=sig)
        return sig
    
    def apply_normalization(self, spec=None):
        if spec is None:
            self.assert_spectrogram()
            self.__spec = preprocessing.normalize(self.__spec)
        else:
            return preprocessing.normalize(spec)
        
    def apply_shift(self, minimum=None):
        self.assert_spectrogram()
        if not minimum:
            self.__spec -= self.__spec.min()
            
    def apply_exp(self, exp):
        self.assert_spectrogram()
        self.__spec = exp**self.__spec
    
    def apply_log10(self):
        self.assert_spectrogram()
        self.__spec = np.log10(self.__spec)
        
    def apply_min_max_scale(self, minimum, maximum, sig=None):
        scale = lambda x: (x-x.min())/(x.max()-x.min())*(maximum-minimum)+minimum
        if sig is None:
            self.assert_spectrogram()
            self.__spec = scale(self.__spec)
        else:
            return scale(sig)
    
    def apply_threshold(self, n, _min=0, _max=None):
        self.assert_spectrogram()
        mean, std = np.mean(self.__spec), np.std(self.__spec)
        width, height = self.__spec.shape
#         _min, _max = self.__spec.min(), self.__spec.max()
        self.__spec = self.__spec.reshape(width*height)
        if _max:
            self.__spec = np.array([_min if x<=mean+n*std else _max for x in self.__spec])
        else:
            self.__spec = np.array([_min if x<=mean+n*std else x+_min for x in self.__spec])
        self.__spec = self.__spec.reshape((width, height))
    
    
    def snr(self, freq_range=[2,10], time_err=0.2, time=0.8):
        """Return the signal to noise ratio of the spectrogram
        params
            freq_range: frequency range of the spectrogram
            time_err: time error used to extract the AWD output
        return
            snr: signal to noise ratio in dB"""
        self.assert_spectrogram()
        # get cuts using the AWD output
        signals, _, _ = self.cuts(cut_freq=freq_range[1]-freq_range[0], cut_freq_min=freq_range[0], cut_time=time, cut_time_split=0.3, time_err=time_err, noise=False)
        signal_power = 0
        area = 0
        # convert spectrogram back to linear scale
        spec = 10**np.copy(self.__spec) 
        noise_signal = np.copy(spec)
        # compute the signal power
        for sig in signals:
            f0, f1, t0, t1 = sig
            signal_power += (spec[f0:f1,t0:t1].flatten()**2).sum()
            area += (t1-t0)*(f1-f0)
            noise_signal[f0:f1,t0:t1].fill(0) 
        # compute the noise power
        noise_power = (noise_signal.flatten()**2).sum()
        noise_power /=(len(self.__time)*len(self.__freq))
        signal_power /= area
        snr = 10*np.log10(signal_power/noise_power)
        return snr
    
    def stats(self, EW=True, spec=False):
        if not spec:
            data = self.frread()[:,0] if EW else self.frread()[:,1]
            data = np.asarray(data)
        else:
            self.assert_spectrogram()
            data = self.__spec.flatten()
        return data.mean(), data.std(), data.min(), data.max(), (data.max()-data.mean())/data.std()
        
    def to_img(self):
        self.assert_spectrogram()
        spec = (self.__spec.astype(np.float64)-self.__spec.min())/(self.__spec.max()-self.__spec.min()) # normalize the data to 0 - 1
        spec = 255*spec
        spec = spec.astype(np.uint8)
        self.__img = Image.fromarray(spec, mode='L').transpose(Image.FLIP_TOP_BOTTOM)
    
    ###############################################
    # SPECTROGRAM VISUALISER
    def spectrogram_plot(self, spec=None, figsize=None, xaxis=False, yaxis=False, xlabel=None, ylabel=None, zlabel=str(__SCALING),
                         title=None, cmap='jet', cmap_show=True,ticks=None, vlines=[], hlines=[], font=[None]*5, output=[]):
        self.assert_spectrogram()
        if spec is None: spec = np.copy(self.__spec);
        fig = plt.figure(figsize=figsize) if figsize else plt.figure()
        for line in vlines:
            plt.axvline(line, color='black', linewidth=font[-1])
        for line in hlines:
            plt.axhline(line, color='black', linewidth=font[-1])
        if xaxis and yaxis:
            img = plt.pcolormesh(self.__time, self.__freq, spec, cmap=cmap)
            if ticks:
                plt.xticks(np.arange(self.__time[0], self.__time[-1], ticks[0]), fontsize=font[3])
                plt.yticks(np.arange(self.__freq[0], self.__freq[-1], ticks[1]), fontsize=font[3])
        else:
            img = plt.pcolormesh(spec, cmap=cmap)
            if ticks:
                plt.xticks(np.arange(0, self.__spec.shape[1], ticks[0]), fontsize=font[3])
                plt.yticks(np.arange(0, self.__spec.shape[0], ticks[1]), fontsize=font[3])
            
        if cmap_show: c = fig.colorbar(mappable=img, label=zlabel);
        if xlabel: plt.xlabel(xlabel, fontsize=font[2]);
        if ylabel: plt.ylabel(ylabel, fontsize=font[2]);
        if title: plt.title(title, fontsize=font[0]);
        for o in output:
            c,x1,x2,y1,y2 = o
            color = 'cornflowerblue'
            rect = patches.Rectangle((x1,y1), x2-x1, y2-y1, linewidth=font[-1], edgecolor=color, facecolor='none')
            plt.text(x1,y2,'%.2f'%c, fontsize=font[-2], bbox={'facecolor':color, 'pad':2, 'ec':color})
            plt.gca().add_patch(rect)
        plt.show()
        
    def spectrogram_hist(self, figsize=None, xlabel='Bins', ylabel=None, title='Histogram'):
        '''Plot histrogram of spectrogram'''
        self.assert_spectrogram()
        if figsize:
            fig = plt.figure(figsize=figsize)
        else:
            fig = plt.figure()
        if xlabel:
            plt.xlabel(xlabel)
        if ylabel:
            plt.ylabel(ylabel)
        if title:
            plt.title(title)
        plt.hist(self.__spec.flatten(), bins=100, histtype='bar', density=True, ec='blue', alpha=0.5)
        plt.show()
        
    def spectrogram_time_cut_plot(self, time, figsize=None, title=None, legend=None, stats=True, legend_loc='upper right'):
        self.assert_spectrogram();
        time_ix = np.round(np.asarray(time)*len(self.__time)/self.__time[-1],0)
        fig = plt.figure(figsize=figsize) if figsize else plt.figure()
        ax = plt.subplot(111)
        ax.margins(0)
        plots, legends = [],legend
        if title:
            plt.title(title)
        for t in time_ix:
            plt.plot(self.__freq, self.__spec[:,int(t)])
            plots.append([self.__freq, self.__spec[:,int(t)]])
        if stats and legend:
            _legend = legend[:]
            for l,t in list(zip(range(len(legend)),time_ix)):
                mean,std = self.__spec[:,int(t)].mean(), self.__spec[:,int(t)].std()
                _legend[l] += ' with mean:'+str(np.round(mean,2))+' and std:'+str(np.round(std,2))
            legends=_legend
        plt.legend(_legend, loc=legend_loc) if legend else plt.legend(time, loc=legend_loc)
        plt.xticks(np.arange(0, self.__freq[-1], 2))
        plt.xlabel('Frequency (kHz)')
        plt.ylabel('Magnitude')
        plt.show()
        return plots, legends

    def spectrogram_freq_cut_plot(self, freq, figsize=None, title=None, legend=None, stats=True, legend_loc='upper right'):
        self.assert_spectrogram();
        freq_ix = np.round(np.asarray(freq)*len(self.__freq)/self.__freq[-1],2)
        fig = plt.figure(figsize=figsize) if figsize else plt.figure()
        ax = plt.subplot(111)
        ax.margins(0)
        if title:
            plt.title(title)
        for f in freq_ix:
            plt.plot(self.__time, self.__spec[int(f),:])
        if stats and legend:
            _legend = legend[:]
            for l,f in list(zip(range(len(legend)),freq_ix)):
                mean,std = self.__spec[int(f),:].mean(), self.__spec[int(f),:].std()
                _legend[l] += ' with mean:'+str(np.round(mean,2))+' and std:'+str(np.round(std,2))
        plt.legend(_legend, loc=legend_loc) if legend else plt.legend(freq, loc=legend_loc)
        plt.xticks(np.arange(0, self.__time[-1], 0.2))
        plt.xlabel('Time (s)')
        plt.ylabel('Magnitude')
        plt.show()
    
    def spectrogram_freq_cum(self, figsize=None, title=None):
        cum = self.__spec[:64,:].sum(axis=0)
        fig = plt.figure(figsize=figsize) if figsize else plt.figure()
        ax = plt.subplot(111)
        ax.margins(0)
        if title:
            plt.title(title)
        plt.plot(self.__time,cum)
#         plt.legend(legend, loc=legend_loc) if legend else plt.legend(freq, loc=legend_loc)
        plt.xlabel('Time (s)')
        plt.show()

    
    ###############################################
    """AUTOMATIC WHISTLERS DETECTOR (AWD) OUTPUT"""
    
    def output(self, verbose=False):
        '''Extract the output information of a file and return the file and the file time format
            as well as the time diffence between the detection of a whistler as per AWDA and the
            begining of the data collection
            returns:
                file time format
                time difference 
        '''
        if self.get_awda_output() is None:
            output_data = self.awda_output(verbose=verbose)[self.__file]
        else:
            output_data = self.get_awda_output()[self.__file]
        file_time = self.__file[:27]
        outputs = []
        for key,value in output_data.items():
            outputs.append([self._datetime_diff(key,file_time), value])
            outputs = sorted(outputs, key=lambda x:x[0])
        self.__output = np.array(outputs)
        return file_time, output_data, self.__output
    
    def awda_whistlers(self, time_err, threshold=0, _output=None):
        """Extract the AWD output based on the time error. Whistler are grouped based on the time_err intervall
        param
            time_err: number of decimal places
            _output: an output format
        return
            output in time
            output in index
        """
        if _output is None:
            _,_, _output = self.output()
        if len(_output)==0:
            return _output
        _output.sort(axis=0)
        # convert to float
        output = np.array([[round(float(o[0]), time_err), float(o[1])] for o in _output if float(o[1])>=threshold])
        uniques = np.sort(np.array(list(set(output[:,0]))))
        t_output = np.array([[u, output[np.argwhere(output[:,0]==u),1].max()] for u in uniques])
        ix_output = np.array([[int(t[0]/self.get_time_res()), int(t[1])] for t in t_output])
        return t_output, ix_output
    
    def awd_starting_locations(self, time_err=1, threshold=10):
        """Get the starting location of the AWD whisltlers"""
        output = self.awda_whistlers(time_err, threshold)[0]
        return np.array([[o[0]-0.2,o[1]] for o in output])
                        
    def output_thresholded(self,threshold):
        '''Return the indexes when the measure of detection is above the threshold'''
        if self.__output is None:
            self.output()
        return [index for index in self.__output if int(index[-1]) >= threshold]
    
    def output_indexed(self, threshold=0, verbose=False):
        self.assert_spectrogram()
        file_time, output_data, _ = self.output(verbose=verbose)
        outputs = self.output_thresholded(threshold)
        _t = np.round_(self.__time,decimals=4)
        index = 0
        indices = []
        indexed_output = []
        for o in outputs:
            event_time = np.round(float(o[0]),4)
            # find index of that event in the spectrogram
            index = min(range(len(_t)), key=lambda i: abs(_t[i]-event_time))
            # only process if the index found is new
            if index!=0 and index not in indices:
                indices.append(index)
            indexed_output.append([index,str(list(output_data.keys())[list(output_data.values()).index(o[1])]),
                                       o[0], o[1]])
        return indexed_output
    
    def output_plot(self, threshold=0, verbose=False, figsize=None, xaxis=None, yaxis=None, xlabel=None, ylabel=None, title=None, cmap='jet'):
        self.assert_spectrogram()
        spectrogram = np.copy(self.__spec)
        # add output markers
        for index_pb in self.output_indexed(threshold, verbose):
            spectrogram[:,index_pb[0]] = np.full(spectrogram[:,index_pb[0]].shape, spectrogram.min())   
        self.spectrogram_plot(spec=spectrogram, figsize=figsize, xaxis=xaxis, yaxis=yaxis, xlabel=xlabel, ylabel=ylabel, title=title, cmap=cmap)
    
    # GENERATE SLICES
    def cuts(self, cut_time=1, cut_time_split=0.2, cut_freq=8, cut_freq_min=1.5, time_err=1, noise=True):
        '''Extract whistler cut out of the spectrogram using the output file oubtained form the AWDA method
        params:
            cut_time: duration of the cut [second]
            cut_freq: frequency span of the cut [kHz]
        returns:
            spectrogram
            whistlers boundaries
            noise boundaries
        '''
        # define slice length
        time_min = cut_time_split*cut_time
        time_max = (1-cut_time_split)*cut_time
        freq_min = cut_freq_min
        freq_max = cut_freq_min+cut_freq
        
        self.assert_spectrogram()
        
        # freqeucny cut indexes
        
        freq_cut_min_index = ceil((freq_min-self.__freq_min)/self.get_freq_res())
        freq_cut_max_index = ceil((freq_max-self.__freq_min)/self.get_freq_res())
#         print('cuts',(freq_min, freq_max) ,(self.get_freq_res()),(freq_cut_min_index, freq_cut_max_index), 
#              (int(freq_min/self.get_freq_res()),int(freq_max/self.get_freq_res())))
#       indices = self.output_indexed(threshold=threshold)
        indices = self.awda_whistlers(time_err=time_err)[1]
        
        time_max_index = self.__time.shape[0] # maximum index on the time axis
        freq_max_index = self.__freq.shape[0] # maximum index on the frequency axis
        time_res = self.get_time_res()   # time resolution
        freq_res = self.get_freq_res() # frequency resolution
        
        whistler_cuts = [] 
        for index in indices:
            # find maximum time for this location
            output_index = int(index[0])
            time_cut_min_index, time_cut_max_index = output_index, output_index
            
            if output_index + int(time_max/time_res) <= time_max_index:
                time_cut_max_index += int(time_max/time_res)
            else:
                extra = abs(time_max_index - (output_index + int(time_max/time_res)))
                time_cut_max_index = time_max_index
                time_cut_min_index -= (int(time_min/time_res)+extra)
                whistler_cuts.append([freq_cut_min_index,freq_cut_max_index,time_cut_min_index,time_cut_max_index])
                continue
            if output_index - int(time_min/time_res) >= 0:
                time_cut_min_index -= int(time_min/time_res)
            else:
                extra = abs(output_index - int(time_min/time_res))
                time_cut_max_index += extra
                time_cut_min_index = 0
            whistler_cuts.append([freq_cut_min_index,freq_cut_max_index,time_cut_min_index,time_cut_max_index])
#             print('whistler: ',(freq_cut_min_index,freq_cut_max_index,time_cut_max_index-time_cut_min_index))
        whistler_cuts = np.array(whistler_cuts)#,dtype=np.float64)
        if noise:
            noise_cuts = []
            # get cuts from time_lower_boundary
            time_cut_min_index = whistler_cuts[:,2].min()
            time_cut_length = int(time_min/time_res)+int(time_max/time_res)#int(cut_time/time_res)
#             print('noise: ', (freq_cut_min_index, freq_cut_max_index,time_cut_length) )
            if time_cut_min_index-time_cut_length > 0:
                for i in range(0, time_cut_min_index, time_cut_length):
                    if time_cut_min_index-i>= time_cut_length:
                        noise_cuts.append([freq_cut_min_index, freq_cut_max_index,i,i+time_cut_length])
            # get cuts from time_upper_boundary
            time_cut_max_index = whistler_cuts[:,3].max()
            if time_cut_max_index+time_cut_length <= time_max_index:
                for i in range(time_cut_max_index, time_max_index, time_cut_length):
                    if i+time_cut_length<=time_max_index:
                        noise_cuts.append([freq_cut_min_index, freq_cut_max_index,i,i+time_cut_length])

            noise_cuts = np.array(noise_cuts)#,dtype=np.float64)
            if len(noise_cuts)>0:
                try:
                    cuts = np.append(whistler_cuts, noise_cuts, axis=0)
                except Exception as e:
                    print('Exception %s at file %s'%(e,self.get_file()))
            else:
                cuts = whistler_cuts
            return cuts, len(indices), len(cuts)
        else:
            return whistler_cuts,len(indices), len(whistler_cuts)
    
    def cut_plot(self, cut):
        spectrogram = self.__spec[cut[0]:cut[1], cut[2]:cut[3]]
        self.spectrogram_plot(spec=self.__spec[cut[0]:cut[1], cut[2]:cut[3]])
    
    def cuts_plot(self, cut_time=1, cut_time_split=0.3, cut_freq=10, cut_freq_min=0,time_err=0, cmap='jet', noise=True, scale=1):
        '''Plot all cuts
        params:
            threshold: threshold for the awda figure of merit
        '''
        cuts, nbr_evt, nbr_cuts = self.cuts(cut_time, cut_time_split,cut_freq,cut_freq_min,time_err=time_err, noise=noise)
        x_size = ceil(sqrt(len(cuts)))
        y_size = ceil(len(cuts)/x_size)
        n = 2 # number of plot per sample
        plt.figure(figsize=(y_size*2*n*scale, x_size*n*scale))
        for plt_ix in range(0,len(cuts)*n, n):
            ax = plt.subplot(x_size, y_size*n, plt_ix+1)
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            ix =int(plt_ix/n)
            cut = cuts[ix]
            plt.pcolormesh(self.__spec[cut[0]:cut[1], cut[2]:cut[3]], cmap=cmap)
            ax.set_title(str(ix)+': '+str(ix<=nbr_evt))
            ax = plt.subplot(x_size, y_size*n, plt_ix+2)
            plt.yticks([])
            plt.grid(False)
            ix =int(plt_ix/n)
            cut = cuts[ix]
            plt.hist(self.__spec[cut[0]:cut[1], cut[2]:cut[3]].flatten(), bins='auto')
            ax.set_title(str(ix)+': '+str(ix<=nbr_evt))
        plt.show()
        
    def split(self, width, height, shape):
        '''Split the spectrogram according to width and height'''
        #     spec = Image.fromarray(spec).resize(shape)
        #     spec = np.array(spec)

        # mark the cut in the spectrogram
        spec = self.__spec.copy()
        f_max, t_max = spec.shape
        t_index = np.array(list(range(0, t_max, ceil(t_max/width))))
        f_index = np.array(list(range(0, f_max, ceil(f_max/height))))
        for t in t_index:
            spec[:,t] = np.full(spec[:,0].shape, spec.min())
        for f in f_index:
            spec[f,:] = np.full(spec[0,:].shape, spec.min())
        # extract the cuts
        cuts = []
        indices = []
        for f in range(len(f_index)):
            for t in range(len(t_index)):
                if t < len(t_index)-1 and f<len(f_index)-1:
                    cut = self.__spec[f_index[f]:f_index[f+1],t_index[t]:t_index[t+1]]
                    indices.append([t_index[t],t_index[t+1],f_index[f],f_index[f+1]])
                elif f == len(f_index)-1 and t < len(t_index)-1:
                    cut = self.__spec[f_index[f]:f_max,t_index[t]:t_index[t+1]]
                    indices.append([t_index[t],t_index[t+1],f_index[f],f_max])
                elif t==len(t_index)-1 and f < len(f_index)-1:
                    cut = self.__spec[f_index[f]:f_index[f+1],t_index[t]:t_max]
                    indices.append([t_index[t],t_max,f_index[f],f_index[f+1]])
                else:
                    cut = self.__spec[f_index[f]:f_max,t_index[t]:t_max]
                    indices.append([t_index[t],t_max,f_index[f],f_max])
                cut = Image.fromarray(cut).resize(shape)
                cut = np.array(cut)
                cuts.append(cut)
        cuts = np.array(cuts)
        return spec, cuts, indices